## From API

In [1]:
import time
import hashlib
import requests
import xml.etree.ElementTree as ET

API_KEY = "N5DHW3ZFJFCXLU8ZBNBY"
API_SECRET = "D^8VB6BAankbhfE96$Gsh2d69MNRT6Gp7azG5X7Q"

URL = "https://api.podcastindex.org/api/1.0/search/byterm?q="

In [ ]:
query = "with"
url = URL + query

print(url)

In [ ]:
epoch_time = int(time.time())
data_to_has = API_KEY + API_SECRET + str(epoch_time)
sha = hashlib.sha1(data_to_has.encode()).hexdigest()

headers = {
    "X-Auth-Date": str(epoch_time),
    "X-Auth-Key": API_KEY,
    "Authorization": sha,
    "User-Agent": "postcasting-index-python-cli",
}

response = requests.get(url, headers=headers)
print(response.json())

In [13]:
data = response.json()

In [ ]:
audio_tracks = []
for podcast_url in data["feeds"]:
    feed_response = requests.get(podcast_url["url"])
    root = ET.fromstring(feed_response.text)
    for item in root.findall(".//item"):
        enclosure = item.find("enclosure")
        if enclosure is not None:
            audio_url = enclosure.get("url")
            audio_tracks.append(audio_url)

print(len(audio_tracks))

## From database

In [7]:
import sqlite3
import xml.etree.ElementTree as ET

In [16]:
conn = sqlite3.connect("podcastindex_feeds.db")
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [9]:
cursor.execute("SELECT COUNT(*) FROM podcasts")
print(cursor.fetchone()[0])

4369929


In [34]:
audio_tracks = []

cursor.execute("SELECT * FROM podcasts LIMIT 10 OFFSET 0;")
for i, row in enumerate(cursor.fetchall()):
    feed_url = dict(row)["url"]
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    feed_response = requests.get(feed_url, headers=headers, timeout=10)

    if feed_response.status_code == 200 and feed_response.content:
        try:
            root = ET.fromstring(feed_response.content)
            for item in root.findall(".//item"):
                enclosure = item.find("enclosure")
                if enclosure is not None:
                    audio_url = enclosure.get("url")
                    if audio_url:
                        audio_tracks.append(audio_url)
        except ET.ParseError as e:
            print(f"XML parsing error for feed {i} {feed_url}: {e}")
    else:
        print(f"Failed to fetch feed {i} {feed_url}")

Failed to fetch feed 5 http://feeds.feedburner.com/DirtBikePodcast


In [35]:
print(len(audio_tracks))

918
